In [1]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
# !pip install -q -U flash_attn --no-build-isolation
!pip uninstall flash_attn -y
!pip install -q -U datasets

!pip install -q -U evaluate
!pip install -q -U tokenizers

!pip install -q diffusers --upgrade
!pip install -q invisible_watermark accelerate safetensors

!pip install -q rouge
!pip install -q rouge_score

!pip install -q bert_score

!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import evaluate
from pprint import pprint
from transformers import AutoConfig

import datasets
import bitsandbytes as bnb
import torch
import random
import pandas as pd
from tqdm import tqdm

import tensorflow as tf
from PIL import Image
import requests

import re
import numpy as np
from scipy.special import softmax

import torch
import transformers
from datasets import Dataset, load_dataset

from transformers import pipeline, BitsAndBytesConfig
from transformers import CLIPProcessor, TFCLIPModel

# For from-scratch T5 model
from transformers import T5TokenizerFast, T5Config, T5ForConditionalGeneration

# For pre-trained T5 model
from transformers import T5Tokenizer, T5ForConditionalGeneration  # this won't import twice, just noting here what's for each model

# For all T5 models
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# For BLEURT (to load a trained model for evaluation)
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# For style classifier model (also for evaluating the seq2seq model output)
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer

In [3]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

# Modify this path to the appropriate location in your Drive
train_dataset = pd.read_excel('drive/MyDrive/266/project/train_data.xlsx')
val_dataset = pd.read_excel('drive/MyDrive/266/project/val_data.xlsx')
test_dataset = pd.read_excel('drive/MyDrive/266/project/test_data.xlsx')

In [5]:
def preprocess_text(df):
  """Convert Excel data to text pairs format"""
  text_pairs = []
  for _, row in df.iterrows():
      line_1 = str(row['line1']).lower()
      line_2 = str(row['line2']).lower()
      text_pairs.append((line_1, line_2))
  return text_pairs

In [6]:
#Let's create some splits
train_pairs = preprocess_text(train_dataset)
val_pairs = preprocess_text(val_dataset)
test_pairs = preprocess_text(test_dataset)

# Final splits
splits = {
    'train': train_pairs,
    'val': val_pairs,
    'test': test_pairs
}

print(f"\nDataset sizes:")
print(f"{len(splits['train'])} training pairs")
print(f"{len(splits['val'])} validation pairs")
print(f"{len(splits['test'])} test pairs")

# Look at some examples
print("Training examples:")
for _ in range(5):
    print(random.choice(train_pairs))


Dataset sizes:
60000 training pairs
15000 validation pairs
15000 test pairs
Training examples:
('always keep the steel on you, yeah, baby', "and they hate it cause i'm still with you, yeah, baby")
("into the hawaiian shorts, relyin' on sports", 'to take us away from here, but i cut too much')
('and i can suck a ten inch cock drier than a raisin', 'brain is amazing, baby')
('17th ward nigga', 'my balls thicker, scars and hard liquor')
('hold up, hold up, wait, yeah', 'i keep like 10 to 12 bitches in here with me')


In [7]:
def make_dataset(pairs, shuffle=False):
    line_1, line_2 = zip(*pairs)
    line_1 = list(line_1)
    line_2 = list(line_2)

    dataset = Dataset.from_dict({"line_1": line_1, "line_2": line_2})
    return dataset.shuffle() if shuffle else dataset

# Usage
train_dataset = make_dataset(splits['train'])
val_dataset = make_dataset(splits['val'])
test_dataset = make_dataset(splits['test'])

In [8]:
# Load the pre-trained model and tokenizer


from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

# Add padding token (GPT-2 doesn't have one by default)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
def tokenize_for_generation(examples):
    # Format as input -> target
    inputs = examples['line_1']
    targets = examples['line_2']

    # Tokenize inputs and targets separately
    model_inputs = tokenizer(inputs, padding=True, truncation=True, max_length=40)
    labels = tokenizer(targets, padding=True, truncation=True, max_length=40)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Add padding token if not already done
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Apply to datasets
train_tokenized = train_dataset.map(tokenize_for_generation, batched=True)
val_tokenized = val_dataset.map(tokenize_for_generation, batched=True)
test_tokenized = test_dataset.map(tokenize_for_generation, batched=True)

# Set format for PyTorch
train_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

In [10]:
# Look at a tokenized example
print("Original:")
print(f"  Input (line_1): {train_dataset[0]['line_1']}")
print(f"  Target (line_2): {train_dataset[0]['line_2']}")
print()
print("Tokenized:")
print(f"  Input IDs: {train_tokenized[0]['input_ids']}")
print(f"  Labels: {train_tokenized[0]['labels']}")
print()
print("Decoded:")
print(f"  Input: {tokenizer.decode(train_tokenized[0]['input_ids'])}")
print(f"  Target: {tokenizer.decode(train_tokenized[0]['labels'])}")

Original:
  Input (line_1): my midwest slabbers - ya dig?
  Target (line_2): ay! yup yup, this is for my east coast slangers

Tokenized:
  Input IDs: tensor([ 1820,  3095,  7038, 38677,  1213,   532, 21349,  3100,    30, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256])
  Labels: tensor([  323,     0,   331,   929,   331,   929,    11,   428,   318,   329,
          616,  7627,  7051,  1017,  6606, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256])

Decoded:
  Input: my midwest slabbers - ya dig?<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftex

In [11]:
def generate_with_pipeline_optimized(first_lines, model, tokenizer):
    """Optimized pipeline generation using Dataset for maximum GPU efficiency"""

    # Set padding side to left for generation
    original_padding_side = tokenizer.padding_side
    tokenizer.padding_side = "left"

    try:
        # Create prompts
        task_prefix = 'Given this rap line, generate the next line: '
        prompts = [task_prefix + line for line in first_lines]

        # # Convert to Dataset
        # dataset = Dataset.from_dict({'text': prompts})

        # Create pipeline
        generator = pipeline(
            'text-generation',
            model=model,
            tokenizer=tokenizer,
            device=0 if torch.cuda.is_available() else -1
        )

        # Generate all at once using the dataset
        outputs = generator(
            prompts,
            max_new_tokens=30,
            temperature=0.8,
            do_sample=True,
            batch_size=16
        )

        raw_texts = []

        # Determine format and extract generated text
        for output in outputs:
            if isinstance(output, dict):
                # Direct dict with 'generated_text' key
                raw_texts.append(output["generated_text"])
            elif isinstance(output, str):
                # Direct string
                raw_texts.append(output)
            elif isinstance(output, list):
                # List of dicts (happens when num_return_sequences > 1 or batch processing)
                if len(output) > 0 and isinstance(output[0], dict):
                    raw_texts.append(output[0]["generated_text"])
                elif len(output) > 0 and isinstance(output[0], str):
                    raw_texts.append(output[0])
                else:
                    raw_texts.append(str(output))
            else:
                # Fallback
                raw_texts.append(str(output))

        # Remove prompt prefix
        generated_lines = [
            raw[len(prompts[i]):].strip()
            for i, raw in enumerate(raw_texts)
        ]

        return generated_lines

    finally:
        # Restore original padding side
        tokenizer.padding_side = original_padding_side

In [12]:
# Extract first lines from your test dataset
test_first_lines = [entry["line_1"] for entry in test_dataset]

print(f"Generating {len(test_first_lines)} examples using optimized pipeline...")

# Generate and store results
generated_lines = generate_with_pipeline_optimized(test_first_lines, model, tokenizer)

test_results = []
for idx in range(len(test_dataset)):
    test_results.append({
        'input': test_dataset[idx]['line_1'],
        'actual': test_dataset[idx]['line_2'],
        'generated': generated_lines[idx]
    })

# Print first 10 results
print("\nFirst 10 Generation Results:")
print("=" * 60)

for i in range(10):
    print(f"\nExample {i+1}:")
    print(f"Input:    {test_results[i]['input']}")
    print(f"Actual:   {test_results[i]['actual']}")
    print(f"Generated:{test_results[i]['generated']}")
    print("-" * 40)

print(f"\nCompleted generation for all {len(test_results)} examples!")

Device set to use cuda:0


Generating 15000 examples using optimized pipeline...

First 10 Generation Results:

Example 1:
Input:    and put my head on his  bible
Actual:   i swear to god i didn't do it
Generated:.
Then, my mother was there and I just did not know how the man would take the sentence.
I then went back to the same
----------------------------------------

Example 2:
Input:    of the picture—punchline, figured out, "ahh, i get you"
Actual:   no, you don't, nigga, so why don't you go'n' figure?
Generated:—and be the next line: of the picture—punchline, figured out, "ahh, i get you"—and be the
----------------------------------------

Example 3:
Input:    got the world followin' the new york script
Actual:   hustle with timbs and hoodies on my new york flip
Generated:into work. We've been working on the project since March 2016 and we've been working on it for a while now. You can find out more
----------------------------------------

Example 4:
Input:    tell it like it is nigga, tell it like it is

In [13]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
nltk.download('punkt')

def calculate_bleu_scores(test_results):
    bleu_scores = []
    smoothing = SmoothingFunction().method1

    for result in test_results:
        reference = [result['actual'].split()]
        candidate = result['generated'].split()

        # Calculate BLEU-4 score
        score = sentence_bleu(reference, candidate, smoothing_function=smoothing)
        bleu_scores.append(score)

    return bleu_scores

# Calculate BLEU scores
bleu_scores = calculate_bleu_scores(test_results)
avg_bleu = sum(bleu_scores) / len(bleu_scores)

print(f"Average BLEU Score: {avg_bleu:.4f}")
print(f"BLEU Score Range: {min(bleu_scores):.4f} - {max(bleu_scores):.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Average BLEU Score: 0.0071
BLEU Score Range: 0.0000 - 1.0000


In [14]:
from rouge import Rouge
from rouge_score import rouge_scorer

def calculate_rouge_scores(test_results):
    """Calculate ROUGE scores, handling empty generations gracefully"""
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    valid_count = 0
    empty_count = 0

    for result in test_results:
        actual = result['actual'].strip()
        generated = result['generated'].strip()

        # Skip if either actual or generated is empty
        if not actual or not generated:
            empty_count += 1
            # Assign 0 score for empty generations
            rouge1_scores.append(0.0)
            rouge2_scores.append(0.0)
            rougeL_scores.append(0.0)
        else:
            valid_count += 1
            scores = rouge_scorer_obj.score(actual, generated)
            rouge1_scores.append(scores['rouge1'].fmeasure)
            rouge2_scores.append(scores['rouge2'].fmeasure)
            rougeL_scores.append(scores['rougeL'].fmeasure)

    print(f"ROUGE Calculation: {valid_count} valid, {empty_count} empty generations")

    return {
        'rouge1': {
            'individual_scores': rouge1_scores,
            'average': np.mean(rouge1_scores),
            'std': np.std(rouge1_scores),
            'valid_count': valid_count,
            'empty_count': empty_count
        },
        'rouge2': {
            'individual_scores': rouge2_scores,
            'average': np.mean(rouge2_scores),
            'std': np.std(rouge2_scores),
            'valid_count': valid_count,
            'empty_count': empty_count
        },
        'rougeL': {
            'individual_scores': rougeL_scores,
            'average': np.mean(rougeL_scores),
            'std': np.std(rougeL_scores),
            'valid_count': valid_count,
            'empty_count': empty_count
        }
    }

# Test the fixed function
rouge_scores = calculate_rouge_scores(test_results)

if rouge_scores:
    print("ROUGE Scores:")
    print(f"  ROUGE-1: {rouge_scores['rouge1']['average']:.4f} (±{rouge_scores['rouge1']['std']:.4f})")
    print(f"  ROUGE-2: {rouge_scores['rouge2']['average']:.4f} (±{rouge_scores['rouge2']['std']:.4f})")
    print(f"  ROUGE-L: {rouge_scores['rougeL']['average']:.4f} (±{rouge_scores['rougeL']['std']:.4f})")

ROUGE Calculation: 14592 valid, 408 empty generations
ROUGE Scores:
  ROUGE-1: 0.0735 (±0.0904)
  ROUGE-2: 0.0105 (±0.0536)
  ROUGE-L: 0.0661 (±0.0830)


In [15]:
# Install BERTScore
!pip install bert-score

from bert_score import score

def calculate_bert_scores(test_results):
    """Calculate BERTScore for all test results"""
    candidates = [result['generated'] for result in test_results]
    references = [result['actual'] for result in test_results]

    # Calculate BERTScore (returns precision, recall, F1)
    P, R, F1 = score(candidates, references, lang="en", verbose=False)

    return {
        'precision': P.mean().item(),
        'recall': R.mean().item(),
        'f1': F1.mean().item(),
        'precision_scores': P.tolist(),
        'recall_scores': R.tolist(),
        'f1_scores': F1.tolist()
    }

# Calculate BERTScore
bert_scores = calculate_bert_scores(test_results)
print(f"BERTScore F1: {bert_scores['f1']:.4f}")
print(f"BERTScore Precision: {bert_scores['precision']:.4f}")
print(f"BERTScore Recall: {bert_scores['recall']:.4f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.7985
BERTScore Precision: 0.7968
BERTScore Recall: 0.8007


In [16]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # 384-dimensional embeddings

def sentence_cosine_similarity(actual_lines, generated_lines):
    """Calculate cosine similarity between whole sentences"""

    # Encode all sentences to get embeddings
    actual_embeddings = model.encode(actual_lines, convert_to_tensor=True)
    generated_embeddings = model.encode(generated_lines, convert_to_tensor=True)

    # Calculate cosine similarity
    cosine_scores = util.pytorch_cos_sim(actual_embeddings, generated_embeddings)

    # Extract diagonal (pairwise similarities)
    similarities = [cosine_scores[i][i].item() for i in range(len(actual_lines))]

    return similarities

# Usage with your test results
actual_lines = [result['actual'] for result in test_results]
generated_lines = [result['generated'] for result in test_results]

sentence_similarities = sentence_cosine_similarity(actual_lines, generated_lines)
avg_sentence_similarity = sum(sentence_similarities) / len(sentence_similarities)

print(f"Average Sentence Cosine Similarity: {avg_sentence_similarity:.4f}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Average Sentence Cosine Similarity: 0.1616


In [17]:
# Install required packages
!pip install bert-score sentence-transformers rouge-score nltk

# Import all necessary libraries
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.corpus import cmudict
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from sentence_transformers import SentenceTransformer, util
# import dandelion.check as rhyme_check
import torch
import numpy as np

# Download required NLTK data
nltk.download('punkt')
nltk.download('cmudict')

class ComprehensiveEvaluator:
    def __init__(self):
        # Initialize Sentence-BERT model
        self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

        # Initialize ROUGE scorer
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

        # Initialize CMU dictionary
        self.cmu_dict = cmudict.dict()

        # Initialize BLEU smoothing function
        self.bleu_smoothing = SmoothingFunction().method1

    def calculate_bleu_scores(self, test_results):
        """Calculate BLEU scores for all test results"""
        bleu_scores = []

        for result in test_results:
            reference = [result['actual'].split()]
            candidate = result['generated'].split()

            if candidate:  # Only calculate if generation is not empty
                score = sentence_bleu(reference, candidate, smoothing_function=self.bleu_smoothing)
                bleu_scores.append(score)
            else:
                bleu_scores.append(0.0)

        return {
            'individual_scores': bleu_scores,
            'average': np.mean(bleu_scores),
            'std': np.std(bleu_scores),
            'min': np.min(bleu_scores),
            'max': np.max(bleu_scores)
        }

    def calculate_rouge_scores(self, test_results):
        """Calculate ROUGE scores for all test results"""
        rouge1_scores = []
        rouge2_scores = []
        rougeL_scores = []

        for result in test_results:
            if result['generated'].strip():  # Only calculate if generation is not empty
                scores = self.rouge_scorer.score(result['actual'], result['generated'])
                rouge1_scores.append(scores['rouge1'].fmeasure)
                rouge2_scores.append(scores['rouge2'].fmeasure)
                rougeL_scores.append(scores['rougeL'].fmeasure)
            else:
                rouge1_scores.append(0.0)
                rouge2_scores.append(0.0)
                rougeL_scores.append(0.0)

        return {
            'rouge1': {
                'individual_scores': rouge1_scores,
                'average': np.mean(rouge1_scores),
                'std': np.std(rouge1_scores)
            },
            'rouge2': {
                'individual_scores': rouge2_scores,
                'average': np.mean(rouge2_scores),
                'std': np.std(rouge2_scores)
            },
            'rougeL': {
                'individual_scores': rougeL_scores,
                'average': np.mean(rougeL_scores),
                'std': np.std(rougeL_scores)
            }
        }

    def calculate_bert_scores(self, test_results):
        """Calculate BERTScore for all test results"""
        candidates = [result['generated'] for result in test_results]
        references = [result['actual'] for result in test_results]

        # Calculate BERTScore
        P, R, F1 = bert_score(candidates, references, lang="en", verbose=False)

        return {
            'precision': {
                'average': P.mean().item(),
                'std': P.std().item(),
                'individual_scores': P.tolist()
            },
            'recall': {
                'average': R.mean().item(),
                'std': R.std().item(),
                'individual_scores': R.tolist()
            },
            'f1': {
                'average': F1.mean().item(),
                'std': F1.std().item(),
                'individual_scores': F1.tolist()
            }
        }

    def calculate_sentence_similarity(self, test_results):
        """Calculate sentence-level cosine similarity using Sentence-BERT"""
        actual_lines = [result['actual'] for result in test_results]
        generated_lines = [result['generated'] for result in test_results]

        # Encode all sentences
        actual_embeddings = self.sentence_model.encode(actual_lines, convert_to_tensor=True)
        generated_embeddings = self.sentence_model.encode(generated_lines, convert_to_tensor=True)

        # Calculate cosine similarity
        cosine_scores = util.pytorch_cos_sim(actual_embeddings, generated_embeddings)

        # Extract diagonal (pairwise similarities)
        similarities = [cosine_scores[i][i].item() for i in range(len(actual_lines))]

        return {
            'individual_scores': similarities,
            'average': np.mean(similarities),
            'std': np.std(similarities),
            'min': np.min(similarities),
            'max': np.max(similarities)
        }

    def get_last_word(self, line):
        """Extract the last word from a line for rhyme analysis"""
        words = line.lower().strip().split()
        if words:
            # Remove punctuation from last word
            last_word = ''.join(c for c in words[-1] if c.isalpha())
            return last_word
        return ""

    def get_rhyme_part_cmu(self, word):
        """Extract the rhyming part using CMU dictionary"""
        if word in self.cmu_dict:
            pronunciations = self.cmu_dict[word]
            if pronunciations:
                # Get the part after the last stressed vowel
                pron = pronunciations[0]
                for i in range(len(pron) - 1, -1, -1):
                    if pron[i][-1].isdigit():  # Stressed vowel
                        return pron[i:]
        return None

    # def analyze_rhymes_dandelion(self, test_results):
    #     """Analyze rhymes using dandelion package"""
    #     perfect_rhymes = 0
    #     vowel_rhymes = 0
    #     total_valid = 0

    #     rhyme_details = []

    #     for i, result in enumerate(test_results):
    #         input_last = self.get_last_word(result['input'])
    #         generated_last = self.get_last_word(result['generated'])

    #         if input_last and generated_last:
    #             total_valid += 1

    #             try:
    #                 # Check for perfect rhymes
    #                 is_perfect = rhyme_check.perfect_rhyme(input_last, generated_last)
    #                 is_vowel = rhyme_check.vowel_rhyme(input_last, generated_last)

    #                 if is_perfect:
    #                     perfect_rhymes += 1
    #                     rhyme_type = 'perfect'
    #                 elif is_vowel:
    #                     vowel_rhymes += 1
    #                     rhyme_type = 'vowel'
    #                 else:
    #                     rhyme_type = 'none'

    #                 rhyme_details.append({
    #                     'example_index': i,
    #                     'input_word': input_last,
    #                     'generated_word': generated_last,
    #                     'rhyme_type': rhyme_type
    #                 })

    #             except Exception as e:
    #                 rhyme_details.append({
    #                     'example_index': i,
    #                     'input_word': input_last,
    #                     'generated_word': generated_last,
    #                     'rhyme_type': 'error',
    #                     'error': str(e)
    #                 })

    #     return {
    #         'perfect_rhyme_rate': perfect_rhymes / total_valid if total_valid > 0 else 0,
    #         'vowel_rhyme_rate': vowel_rhymes / total_valid if total_valid > 0 else 0,
    #         'total_rhyme_rate': (perfect_rhymes + vowel_rhymes) / total_valid if total_valid > 0 else 0,
    #         'perfect_rhymes': perfect_rhymes,
    #         'vowel_rhymes': vowel_rhymes,
    #         'total_valid': total_valid,
    #         'details': rhyme_details
    #     }

    def analyze_rhymes_cmu(self, test_results):
        """Analyze rhymes using CMU dictionary"""
        phonetic_rhymes = 0
        total_valid = 0

        rhyme_details = []

        for i, result in enumerate(test_results):
            input_last = self.get_last_word(result['input'])
            generated_last = self.get_last_word(result['generated'])

            if input_last and generated_last:
                input_rhyme = self.get_rhyme_part_cmu(input_last)
                generated_rhyme = self.get_rhyme_part_cmu(generated_last)

                if input_rhyme and generated_rhyme:
                    total_valid += 1

                    is_rhyme = input_rhyme == generated_rhyme
                    if is_rhyme:
                        phonetic_rhymes += 1

                    rhyme_details.append({
                        'example_index': i,
                        'input_word': input_last,
                        'generated_word': generated_last,
                        'input_phonemes': input_rhyme,
                        'generated_phonemes': generated_rhyme,
                        'is_rhyme': is_rhyme
                    })

        return {
            'phonetic_rhyme_rate': phonetic_rhymes / total_valid if total_valid > 0 else 0,
            'phonetic_rhymes': phonetic_rhymes,
            'total_valid': total_valid,
            'details': rhyme_details
        }

    def calculate_length_similarity(self, test_results):
        """Calculate length similarity between actual and generated lines"""
        length_diffs = []
        length_ratios = []

        for result in test_results:
            actual_len = len(result['actual'].split())
            generated_len = len(result['generated'].split())

            length_diffs.append(abs(actual_len - generated_len))

            if actual_len > 0:
                length_ratios.append(generated_len / actual_len)
            else:
                length_ratios.append(0.0)

        return {
            'average_length_diff': np.mean(length_diffs),
            'std_length_diff': np.std(length_diffs),
            'average_length_ratio': np.mean(length_ratios),
            'std_length_ratio': np.std(length_ratios)
        }

    def evaluate_comprehensive(self, test_results):
        """Run comprehensive evaluation on test results"""
        print("=" * 80)
        print("COMPREHENSIVE EVALUATION RESULTS")
        print("=" * 80)

        # Basic statistics
        total_examples = len(test_results)
        empty_generations = sum(1 for r in test_results if not r['generated'].strip())

        print(f"Dataset Statistics:")
        print(f"  Total Examples: {total_examples}")
        print(f"  Empty Generations: {empty_generations} ({empty_generations/total_examples:.1%})")
        print()

        # Calculate all metrics
        print("Computing metrics...")

        # Traditional NLP metrics
        bleu_results = self.calculate_bleu_scores(test_results)
        rouge_results = self.calculate_rouge_scores(test_results)
        bert_results = self.calculate_bert_scores(test_results)

        # Sentence-level similarity
        sentence_sim_results = self.calculate_sentence_similarity(test_results)

        # Rhyme analysis
        # dandelion_rhyme_results = self.analyze_rhymes_dandelion(test_results)
        cmu_rhyme_results = self.analyze_rhymes_cmu(test_results)

        # Length analysis
        length_results = self.calculate_length_similarity(test_results)

        # Display results
        print("\n" + "="*60)
        print("TRADITIONAL NLP METRICS")
        print("="*60)

        print(f"BLEU Score:")
        print(f"  Average: {bleu_results['average']:.4f} (±{bleu_results['std']:.4f})")
        print(f"  Range: {bleu_results['min']:.4f} - {bleu_results['max']:.4f}")

        print(f"\nROUGE Scores:")
        print(f"  ROUGE-1: {rouge_results['rouge1']['average']:.4f} (±{rouge_results['rouge1']['std']:.4f})")
        print(f"  ROUGE-2: {rouge_results['rouge2']['average']:.4f} (±{rouge_results['rouge2']['std']:.4f})")
        print(f"  ROUGE-L: {rouge_results['rougeL']['average']:.4f} (±{rouge_results['rougeL']['std']:.4f})")

        print(f"\nBERTScore:")
        print(f"  F1: {bert_results['f1']['average']:.4f} (±{bert_results['f1']['std']:.4f})")
        print(f"  Precision: {bert_results['precision']['average']:.4f} (±{bert_results['precision']['std']:.4f})")
        print(f"  Recall: {bert_results['recall']['average']:.4f} (±{bert_results['recall']['std']:.4f})")

        print("\n" + "="*60)
        print("SENTENCE-LEVEL SEMANTIC SIMILARITY")
        print("="*60)

        print(f"Sentence-BERT Cosine Similarity:")
        print(f"  Average: {sentence_sim_results['average']:.4f} (±{sentence_sim_results['std']:.4f})")
        print(f"  Range: {sentence_sim_results['min']:.4f} - {sentence_sim_results['max']:.4f}")

        print("\n" + "="*60)
        print("RHYME ANALYSIS")
        print("="*60)

        # print(f"Dandelion Rhyme Analysis:")
        # print(f"  Perfect Rhyme Rate: {dandelion_rhyme_results['perfect_rhyme_rate']:.2%}")
        # print(f"  Vowel Rhyme Rate: {dandelion_rhyme_results['vowel_rhyme_rate']:.2%}")
        # print(f"  Total Rhyme Rate: {dandelion_rhyme_results['total_rhyme_rate']:.2%}")
        # print(f"  Valid Examples: {dandelion_rhyme_results['total_valid']}/{total_examples}")

        print(f"\nCMU Dictionary Phonetic Analysis:")
        print(f"  Phonetic Rhyme Rate: {cmu_rhyme_results['phonetic_rhyme_rate']:.2%}")
        print(f"  Valid Examples: {cmu_rhyme_results['total_valid']}/{total_examples}")

        print("\n" + "="*60)
        print("LENGTH ANALYSIS")
        print("="*60)

        print(f"Length Similarity:")
        print(f"  Average Length Difference: {length_results['average_length_diff']:.2f} words")
        print(f"  Average Length Ratio: {length_results['average_length_ratio']:.2f}")

        print("=" * 80)

        # Return all results for further analysis
        return {
            'basic_stats': {
                'total_examples': total_examples,
                'empty_generations': empty_generations
            },
            'bleu': bleu_results,
            'rouge': rouge_results,
            'bert_score': bert_results,
            'sentence_similarity': sentence_sim_results,
            # 'dandelion_rhyme': dandelion_rhyme_results,
            'cmu_rhyme': cmu_rhyme_results,
            'length_analysis': length_results
        }

# Usage
evaluator = ComprehensiveEvaluator()
comprehensive_results = evaluator.evaluate_comprehensive(test_results)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


COMPREHENSIVE EVALUATION RESULTS
Dataset Statistics:
  Total Examples: 15000
  Empty Generations: 408 (2.7%)

Computing metrics...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



TRADITIONAL NLP METRICS
BLEU Score:
  Average: 0.0071 (±0.0271)
  Range: 0.0000 - 1.0000

ROUGE Scores:
  ROUGE-1: 0.0735 (±0.0904)
  ROUGE-2: 0.0105 (±0.0536)
  ROUGE-L: 0.0661 (±0.0830)

BERTScore:
  F1: 0.7985 (±0.1354)
  Precision: 0.7968 (±0.1362)
  Recall: 0.8007 (±0.1364)

SENTENCE-LEVEL SEMANTIC SIMILARITY
Sentence-BERT Cosine Similarity:
  Average: 0.1616 (±0.1208)
  Range: -0.1368 - 1.0000

RHYME ANALYSIS

CMU Dictionary Phonetic Analysis:
  Phonetic Rhyme Rate: 4.53%
  Valid Examples: 11229/15000

LENGTH ANALYSIS
Length Similarity:
  Average Length Difference: 11.62 words
  Average Length Ratio: 2.74


In [18]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.corpus import cmudict
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import time
from typing import Dict, List, Any
import warnings

# Download required NLTK data
nltk.download('punkt', quiet=True)
nltk.download('cmudict', quiet=True)

class ComprehensiveEvaluator:
    def __init__(self, sentence_model_name: str = 'all-MiniLM-L6-v2', device: str = None):
        """
        Initialize evaluator with configurable models and device

        Args:
            sentence_model_name: Name of sentence transformer model to use
            device: Device to run models on ('cuda', 'cpu', or None for auto)
        """
        # Set device
        if device is None:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            self.device = device

        print(f"Initializing evaluator on device: {self.device}")

        # Initialize models with error handling
        try:
            self.sentence_model = SentenceTransformer(sentence_model_name, device=self.device)
            print(f"✓ Loaded Sentence-BERT model: {sentence_model_name}")
        except Exception as e:
            warnings.warn(f"Failed to load Sentence-BERT model: {e}")
            self.sentence_model = None

        # Initialize ROUGE scorer
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

        # Initialize CMU dictionary with error handling
        try:
            self.cmu_dict = cmudict.dict()
            print(f"✓ Loaded CMU dictionary with {len(self.cmu_dict)} entries")
        except Exception as e:
            warnings.warn(f"Failed to load CMU dictionary: {e}")
            self.cmu_dict = {}

        # Initialize BLEU smoothing function
        self.bleu_smoothing = SmoothingFunction().method1

        # Cache for performance
        self._rhyme_cache = {}

    def calculate_bleu_scores(self, test_results: List[Dict]) -> Dict[str, Any]:
        """Calculate BLEU scores with improved handling"""
        bleu_scores = []
        valid_count = 0
        empty_count = 0

        for result in test_results:
            actual = result['actual'].strip()
            generated = result['generated'].strip()

            if not generated:
                empty_count += 1
                bleu_scores.append(0.0)
            elif not actual:
                empty_count += 1
                bleu_scores.append(0.0)
            else:
                valid_count += 1
                reference = [actual.split()]
                candidate = generated.split()

                score = sentence_bleu(reference, candidate, smoothing_function=self.bleu_smoothing)
                bleu_scores.append(score)

        return {
            'individual_scores': bleu_scores,
            'average': np.mean(bleu_scores),
            'std': np.std(bleu_scores),
            'min': np.min(bleu_scores),
            'max': np.max(bleu_scores),
            'valid_count': valid_count,
            'empty_count': empty_count
        }

    def calculate_rouge_scores(self, test_results: List[Dict]) -> Dict[str, Any]:
        """Calculate ROUGE scores with enhanced tracking"""
        rouge1_scores = []
        rouge2_scores = []
        rougeL_scores = []

        valid_count = 0
        empty_count = 0

        for result in test_results:
            actual = result['actual'].strip()
            generated = result['generated'].strip()

            if not generated or not actual:
                empty_count += 1
                rouge1_scores.append(0.0)
                rouge2_scores.append(0.0)
                rougeL_scores.append(0.0)
            else:
                valid_count += 1
                scores = self.rouge_scorer.score(actual, generated)
                rouge1_scores.append(scores['rouge1'].fmeasure)
                rouge2_scores.append(scores['rouge2'].fmeasure)
                rougeL_scores.append(scores['rougeL'].fmeasure)

        return {
            'rouge1': {
                'individual_scores': rouge1_scores,
                'average': np.mean(rouge1_scores),
                'std': np.std(rouge1_scores),
                'valid_count': valid_count,
                'empty_count': empty_count
            },
            'rouge2': {
                'individual_scores': rouge2_scores,
                'average': np.mean(rouge2_scores),
                'std': np.std(rouge2_scores),
                'valid_count': valid_count,
                'empty_count': empty_count
            },
            'rougeL': {
                'individual_scores': rougeL_scores,
                'average': np.mean(rougeL_scores),
                'std': np.std(rougeL_scores),
                'valid_count': valid_count,
                'empty_count': empty_count
            }
        }

    def calculate_bert_scores(self, test_results: List[Dict]) -> Dict[str, Any]:
        """Calculate BERTScore with batch processing"""
        candidates = [result['generated'] for result in test_results]
        references = [result['actual'] for result in test_results]

        try:
            # Calculate BERTScore with device specification
            P, R, F1 = bert_score(candidates, references, lang="en", verbose=False, device=self.device)

            return {
                'precision': {
                    'average': P.mean().item(),
                    'std': P.std().item(),
                    'individual_scores': P.tolist()
                },
                'recall': {
                    'average': R.mean().item(),
                    'std': R.std().item(),
                    'individual_scores': R.tolist()
                },
                'f1': {
                    'average': F1.mean().item(),
                    'std': F1.std().item(),
                    'individual_scores': F1.tolist()
                }
            }
        except Exception as e:
            warnings.warn(f"BERTScore calculation failed: {e}")
            return None

    def calculate_sentence_similarity(self, test_results: List[Dict]) -> Dict[str, Any]:
        """Calculate sentence similarity with error handling"""
        if self.sentence_model is None:
            warnings.warn("Sentence-BERT model not available")
            return None

        actual_lines = [result['actual'] for result in test_results]
        generated_lines = [result['generated'] for result in test_results]

        try:
            # Encode all sentences with batch processing
            actual_embeddings = self.sentence_model.encode(actual_lines, convert_to_tensor=True, show_progress_bar=False)
            generated_embeddings = self.sentence_model.encode(generated_lines, convert_to_tensor=True, show_progress_bar=False)

            # Calculate cosine similarity
            cosine_scores = util.pytorch_cos_sim(actual_embeddings, generated_embeddings)

            # Extract diagonal (pairwise similarities)
            similarities = [cosine_scores[i][i].item() for i in range(len(actual_lines))]

            return {
                'individual_scores': similarities,
                'average': np.mean(similarities),
                'std': np.std(similarities),
                'min': np.min(similarities),
                'max': np.max(similarities)
            }
        except Exception as e:
            warnings.warn(f"Sentence similarity calculation failed: {e}")
            return None

    def get_last_word(self, line: str) -> str:
        """Extract last word with improved cleaning"""
        import re
        # Use regex to better handle punctuation and contractions
        words = re.findall(r"\b[a-zA-Z]+(?:'[a-zA-Z]+)?\b", line.lower())
        return words[-1] if words else ""

    def get_rhyme_part_cmu(self, word: str) -> List[str]:
        """Extract rhyming part with caching"""
        if word in self._rhyme_cache:
            return self._rhyme_cache[word]

        if word in self.cmu_dict:
            pronunciations = self.cmu_dict[word]
            if pronunciations:
                # Get the part after the last stressed vowel
                pron = pronunciations[0]
                for i in range(len(pron) - 1, -1, -1):
                    if pron[i][-1].isdigit():  # Stressed vowel
                        result = pron[i:]
                        self._rhyme_cache[word] = result
                        return result

        self._rhyme_cache[word] = None
        return None

    def analyze_rhymes_cmu(self, test_results: List[Dict]) -> Dict[str, Any]:
        """Enhanced rhyme analysis with better statistics"""
        phonetic_rhymes = 0
        near_rhymes = 0  # Rhymes with similar endings
        total_valid = 0
        total_processed = 0

        rhyme_details = []

        for i, result in enumerate(test_results):
            input_last = self.get_last_word(result['input'])
            generated_last = self.get_last_word(result['generated'])

            total_processed += 1

            if input_last and generated_last:
                input_rhyme = self.get_rhyme_part_cmu(input_last)
                generated_rhyme = self.get_rhyme_part_cmu(generated_last)

                if input_rhyme and generated_rhyme:
                    total_valid += 1

                    is_perfect_rhyme = input_rhyme == generated_rhyme
                    is_near_rhyme = False

                    # Check for near rhymes (last 2 phonemes match)
                    if not is_perfect_rhyme and len(input_rhyme) >= 2 and len(generated_rhyme) >= 2:
                        is_near_rhyme = input_rhyme[-2:] == generated_rhyme[-2:]

                    if is_perfect_rhyme:
                        phonetic_rhymes += 1
                    elif is_near_rhyme:
                        near_rhymes += 1

                    rhyme_details.append({
                        'example_index': i,
                        'input_word': input_last,
                        'generated_word': generated_last,
                        'input_phonemes': input_rhyme,
                        'generated_phonemes': generated_rhyme,
                        'is_perfect_rhyme': is_perfect_rhyme,
                        'is_near_rhyme': is_near_rhyme
                    })

        return {
            'perfect_rhyme_rate': phonetic_rhymes / total_valid if total_valid > 0 else 0,
            'near_rhyme_rate': near_rhymes / total_valid if total_valid > 0 else 0,
            'total_rhyme_rate': (phonetic_rhymes + near_rhymes) / total_valid if total_valid > 0 else 0,
            'perfect_rhymes': phonetic_rhymes,
            'near_rhymes': near_rhymes,
            'total_valid': total_valid,
            'total_processed': total_processed,
            'coverage': total_valid / total_processed if total_processed > 0 else 0,
            'details': rhyme_details
        }

    def calculate_additional_metrics(self, test_results: List[Dict]) -> Dict[str, Any]:
        """Calculate additional rap-specific metrics"""

        # Syllable analysis (approximate)
        def count_syllables(word):
            # Simple syllable counting heuristic
            word = word.lower()
            count = 0
            vowels = "aeiouy"
            if word[0] in vowels:
                count += 1
            for i in range(1, len(word)):
                if word[i] in vowels and word[i-1] not in vowels:
                    count += 1
            if word.endswith("e"):
                count -= 1
            if count == 0:
                count += 1
            return count

        syllable_diffs = []
        word_diversity_scores = []

        for result in test_results:
            actual_words = result['actual'].split()
            generated_words = result['generated'].split()

            # Syllable analysis
            if actual_words and generated_words:
                actual_syllables = sum(count_syllables(word) for word in actual_words)
                generated_syllables = sum(count_syllables(word) for word in generated_words)
                syllable_diffs.append(abs(actual_syllables - generated_syllables))
            else:
                syllable_diffs.append(0)

            # Word diversity (unique words / total words)
            if generated_words:
                diversity = len(set(generated_words)) / len(generated_words)
                word_diversity_scores.append(diversity)
            else:
                word_diversity_scores.append(0.0)

        return {
            'syllable_similarity': {
                'average_diff': np.mean(syllable_diffs),
                'std_diff': np.std(syllable_diffs)
            },
            'word_diversity': {
                'average': np.mean(word_diversity_scores),
                'std': np.std(word_diversity_scores)
            }
        }

    def evaluate_comprehensive(self, test_results: List[Dict], show_progress: bool = True) -> Dict[str, Any]:
        """Enhanced comprehensive evaluation with timing and progress"""
        start_time = time.time()

        print("=" * 80)
        print("COMPREHENSIVE EVALUATION RESULTS")
        print("=" * 80)

        # Basic statistics
        total_examples = len(test_results)
        empty_generations = sum(1 for r in test_results if not r['generated'].strip())

        print(f"Dataset Statistics:")
        print(f"  Total Examples: {total_examples}")
        print(f"  Empty Generations: {empty_generations} ({empty_generations/total_examples:.1%})")
        print()

        # Calculate all metrics with timing
        results = {}

        if show_progress:
            print("Computing metrics...")

        # Traditional NLP metrics
        if show_progress: print("  • BLEU scores...")
        results['bleu'] = self.calculate_bleu_scores(test_results)

        if show_progress: print("  • ROUGE scores...")
        results['rouge'] = self.calculate_rouge_scores(test_results)

        if show_progress: print("  • BERTScore...")
        results['bert_score'] = self.calculate_bert_scores(test_results)

        # Sentence-level similarity
        if show_progress: print("  • Sentence similarity...")
        results['sentence_similarity'] = self.calculate_sentence_similarity(test_results)

        # Rhyme analysis
        if show_progress: print("  • Rhyme analysis...")
        results['cmu_rhyme'] = self.analyze_rhymes_cmu(test_results)

        # Additional metrics
        if show_progress: print("  • Additional metrics...")
        results['additional_metrics'] = self.calculate_additional_metrics(test_results)

        # Length analysis
        results['length_analysis'] = self.calculate_length_similarity(test_results)

        # Display results with enhanced formatting
        self._display_results(results, total_examples, empty_generations)

        execution_time = time.time() - start_time
        print(f"\nEvaluation completed in {execution_time:.2f} seconds")
        print("=" * 80)

        # Add metadata
        results['metadata'] = {
            'total_examples': total_examples,
            'empty_generations': empty_generations,
            'execution_time': execution_time,
            'device_used': self.device
        }

        return results

    def _display_results(self, results: Dict, total_examples: int, empty_generations: int):
        """Enhanced result display with better formatting"""

        print("\n" + "="*60)
        print("TRADITIONAL NLP METRICS")
        print("="*60)

        # BLEU
        bleu = results['bleu']
        print(f"BLEU Score:")
        print(f"  Average: {bleu['average']:.4f} (±{bleu['std']:.4f})")
        print(f"  Range: {bleu['min']:.4f} - {bleu['max']:.4f}")
        print(f"  Valid/Empty: {bleu['valid_count']}/{bleu['empty_count']}")

        # ROUGE
        rouge = results['rouge']
        print(f"\nROUGE Scores:")
        print(f"  ROUGE-1: {rouge['rouge1']['average']:.4f} (±{rouge['rouge1']['std']:.4f})")
        print(f"  ROUGE-2: {rouge['rouge2']['average']:.4f} (±{rouge['rouge2']['std']:.4f})")
        print(f"  ROUGE-L: {rouge['rougeL']['average']:.4f} (±{rouge['rougeL']['std']:.4f})")
        print(f"  Valid/Empty: {rouge['rouge1']['valid_count']}/{rouge['rouge1']['empty_count']}")

        # BERTScore
        if results['bert_score']:
            bert = results['bert_score']
            print(f"\nBERTScore:")
            print(f"  F1: {bert['f1']['average']:.4f} (±{bert['f1']['std']:.4f})")
            print(f"  Precision: {bert['precision']['average']:.4f} (±{bert['precision']['std']:.4f})")
            print(f"  Recall: {bert['recall']['average']:.4f} (±{bert['recall']['std']:.4f})")

        # Sentence similarity
        if results['sentence_similarity']:
            sent_sim = results['sentence_similarity']
            print("\n" + "="*60)
            print("SENTENCE-LEVEL SEMANTIC SIMILARITY")
            print("="*60)
            print(f"Sentence-BERT Cosine Similarity:")
            print(f"  Average: {sent_sim['average']:.4f} (±{sent_sim['std']:.4f})")
            print(f"  Range: {sent_sim['min']:.4f} - {sent_sim['max']:.4f}")

        # Rhyme analysis
        rhyme = results['cmu_rhyme']
        print("\n" + "="*60)
        print("RHYME ANALYSIS")
        print("="*60)
        print(f"CMU Dictionary Phonetic Analysis:")
        print(f"  Perfect Rhyme Rate: {rhyme['perfect_rhyme_rate']:.2%}")
        print(f"  Near Rhyme Rate: {rhyme['near_rhyme_rate']:.2%}")
        print(f"  Total Rhyme Rate: {rhyme['total_rhyme_rate']:.2%}")
        print(f"  Dictionary Coverage: {rhyme['coverage']:.1%} ({rhyme['total_valid']}/{rhyme['total_processed']})")

        # Additional metrics
        additional = results['additional_metrics']
        print("\n" + "="*60)
        print("RAP-SPECIFIC METRICS")
        print("="*60)
        print(f"Syllable Similarity:")
        print(f"  Average Difference: {additional['syllable_similarity']['average_diff']:.2f} syllables")
        print(f"Word Diversity:")
        print(f"  Average: {additional['word_diversity']['average']:.3f}")

        # Length analysis
        length = results['length_analysis']
        print(f"\nLength Analysis:")
        print(f"  Average Length Difference: {length['average_length_diff']:.2f} words")
        print(f"  Average Length Ratio: {length['average_length_ratio']:.2f}")

    def calculate_length_similarity(self, test_results: List[Dict]) -> Dict[str, Any]:
        """Enhanced length analysis"""
        length_diffs = []
        length_ratios = []

        for result in test_results:
            actual_len = len(result['actual'].split())
            generated_len = len(result['generated'].split())

            length_diffs.append(abs(actual_len - generated_len))

            if actual_len > 0:
                length_ratios.append(generated_len / actual_len)
            else:
                length_ratios.append(0.0)

        return {
            'average_length_diff': np.mean(length_diffs),
            'std_length_diff': np.std(length_diffs),
            'average_length_ratio': np.mean(length_ratios),
            'std_length_ratio': np.std(length_ratios),
            'individual_diffs': length_diffs,
            'individual_ratios': length_ratios
        }

In [19]:
# Initialize with custom settings
evaluator = ComprehensiveEvaluator(
    sentence_model_name='all-MiniLM-L6-v2',  # or 'all-mpnet-base-v2' for better quality
    device='cuda'  # or 'cpu'
)

# Run evaluation
comprehensive_results = evaluator.evaluate_comprehensive(test_results, show_progress=True)

Initializing evaluator on device: cuda
✓ Loaded Sentence-BERT model: all-MiniLM-L6-v2
✓ Loaded CMU dictionary with 123455 entries
COMPREHENSIVE EVALUATION RESULTS
Dataset Statistics:
  Total Examples: 15000
  Empty Generations: 408 (2.7%)

Computing metrics...
  • BLEU scores...
  • ROUGE scores...
  • BERTScore...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  • Sentence similarity...
  • Rhyme analysis...
  • Additional metrics...

TRADITIONAL NLP METRICS
BLEU Score:
  Average: 0.0071 (±0.0271)
  Range: 0.0000 - 1.0000
  Valid/Empty: 14592/408

ROUGE Scores:
  ROUGE-1: 0.0735 (±0.0904)
  ROUGE-2: 0.0105 (±0.0536)
  ROUGE-L: 0.0661 (±0.0830)
  Valid/Empty: 14592/408

BERTScore:
  F1: 0.7985 (±0.1354)
  Precision: 0.7968 (±0.1362)
  Recall: 0.8007 (±0.1364)

SENTENCE-LEVEL SEMANTIC SIMILARITY
Sentence-BERT Cosine Similarity:
  Average: 0.1616 (±0.1208)
  Range: -0.1368 - 1.0000

RHYME ANALYSIS
CMU Dictionary Phonetic Analysis:
  Perfect Rhyme Rate: 4.52%
  Near Rhyme Rate: 0.07%
  Total Rhyme Rate: 4.60%
  Dictionary Coverage: 80.9% (12137/15000)

RAP-SPECIFIC METRICS
Syllable Similarity:
  Average Difference: 13.50 syllables
Word Diversity:
  Average: 0.824

Length Analysis:
  Average Length Difference: 11.62 words
  Average Length Ratio: 2.74

Evaluation completed in 154.30 seconds
